[link text](https://)# Module 5 — Transfer Learning & Famous Architectures (Expanded)

This expanded notebook demonstrates practical transfer learning workflows using ResNet50 and MobileNetV2 on CIFAR-10 (small subset for classroom demos). It includes:

- dataset download and preprocessing
- augmentation pipeline
- feature-extraction (freeze backbone) training
- fine-tuning (unfreeze top layers)
- evaluation and model saving

Notes: Use GPU runtime in Colab for faster training. Training epochs are kept small for demos.

## 1 — Setup (install packages and imports)

In [ ]:
# Install / ensure tensorflow
!pip -q install -U tensorflow==2.12.0 --quiet

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
print('TF version:', tf.__version__)
print('GPU available:', tf.config.list_physical_devices('GPU'))


## 2 — Load CIFAR-10 and prepare datasets (with augmentation)

In [ ]:
from tensorflow.keras.datasets import cifar10
(x_train,y_train),(x_test,y_test)=cifar10.load_data()
# reduce size for demo speed
x_train = x_train[:10000].astype('float32')
y_train = y_train[:10000]
x_test = x_test[:2000].astype('float32')
y_test = y_test[:2000]

# Resize to backbone input size (224x224) and build tf.data
IMG_SIZE = 224
BATCH = 32
AUTOTUNE = tf.data.AUTOTUNE

def preprocess(image, label, training=False):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    if training:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_brightness(image, 0.1)
        image = tf.image.random_contrast(image, 0.9, 1.1)
    return image, label

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(lambda x,y: preprocess(x,y,training=True), num_parallel_calls=AUTOTUNE).shuffle(2000).batch(BATCH).prefetch(AUTOTUNE)
val_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).map(lambda x,y: preprocess(x,y,training=False), num_parallel_calls=AUTOTUNE).batch(BATCH).prefetch(AUTOTUNE)

print('Sample batch shapes:')
for imgs, labs in train_ds.take(1):
    print(imgs.shape, labs.shape)


## 3 — Build transfer learning model helper (ResNet50 / MobileNetV2)

In [ ]:
from tensorflow.keras.applications import ResNet50, MobileNetV2
from tensorflow.keras import layers, models

NUM_CLASSES = 10

def build_transfer_model(backbone='resnet50', img_size=IMG_SIZE, num_classes=NUM_CLASSES):
    if backbone == 'resnet50':
        base = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size,img_size,3))
    elif backbone == 'mobilenetv2':
        base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size,img_size,3))
    else:
        raise ValueError('Unsupported backbone')
    base.trainable = False
    inputs = layers.Input(shape=(img_size,img_size,3))
    x = base(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build ResNet model
model_resnet = build_transfer_model('resnet50')
model_resnet.summary()


## 4 — Train classifier head (feature extraction) — ResNet50 example

In [ ]:
# Train for a few epochs to demonstrate feature extraction
history_resnet = model_resnet.fit(train_ds, validation_data=val_ds, epochs=3)
plt.figure(figsize=(8,4))
plt.plot(history_resnet.history['val_accuracy'], label='val_acc')
plt.plot(history_resnet.history['accuracy'], label='train_acc')
plt.legend(); plt.title('ResNet feature-extractor training (few epochs)')
plt.show()


## 5 — Evaluate ResNet feature-extractor

In [ ]:
loss, acc = model_resnet.evaluate(val_ds)
print('ResNet val loss, acc:', loss, acc)


## 6 — Fine-tune: unfreeze top layers and continue training (ResNet)

In [ ]:
# Unfreeze last blocks for fine-tuning
base_model = model_resnet.layers[1]  # assuming base is second layer
# Safely set trainable for top layers
for layer in base_model.layers:
    layer.trainable = False
# Unfreeze last N layers
N = 30
for layer in base_model.layers[-N:]:
    layer.trainable = True

model_resnet.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# fine-tune for a few epochs
history_fine = model_resnet.fit(train_ds, validation_data=val_ds, epochs=2)
plt.plot(history_fine.history['val_accuracy'], label='val_acc_fine')
plt.legend(); plt.title('Fine-tuning')
plt.show()


## 7 — Repeat with MobileNetV2 (feature extraction)

In [ ]:
model_mobilenet = build_transfer_model('mobilenetv2')
history_mn = model_mobilenet.fit(train_ds, validation_data=val_ds, epochs=2)
print('MobileNetV2 training done')


## 8 — Save models and tips for export

In [ ]:
model_resnet.save('/mnt/data/resnet50_cifar10_head.h5')
model_mobilenet.save('/mnt/data/mobilenetv2_cifar10_head.h5')
print('Saved models to /mnt/data')

# Quick note: to export to TFLite, use tf.lite.TFLiteConverter.from_keras_model(model)


## 9 — Instructor exercises & notes

- Try training longer and compare ResNet50 vs MobileNetV2 accuracy and training time.
- Observe memory / GPU usage: MobileNet is lighter and faster for edge demos.
- For class imbalance, demonstrate class weights or oversampling.
- Export model to SavedModel and TFLite for deployment demos.